## Importing relevant libraries and files

In [1]:
import pandas as pd
import numpy as np

In [2]:
#Importing labels and raw data file
YK_label = pd.read_excel(r'YK_LABEL.xlsx')
YK_data = pd.read_csv('YK-RawNew.csv')

ImportError: Missing optional dependency 'openpyxl'.  Use pip or conda to install openpyxl.

In [3]:
#Looking at the shape of each dataset
print(YK_data.shape)

(69577, 1076)


The dimensions of each file describes the data. The first element of the shape tuple corresponds to the number of rows (i.e the number of data points that needs to be downsampled to 50,000), while the second element of the tuple represents the number of columns (i.e the number of tests conducted)

In [4]:
#Function to drop columns (tests) in the raw data file which are NOT tests (i.e metadata like start time, lot number etc)
def test_only(datafile,labelfile):
    keys = list(labelfile['Name'])
    df = datafile[keys]
    return df

In [5]:
#Applying the functions to file 
YK_dropped = test_only(YK_data,YK_label)
#Checking the shape and how the data looks like
print(YK_dropped.shape)
print(YK_dropped.head())

(69577, 1039)
   temperature  Device_Test_Time  \
0         25.0           22332.0   
1         25.0           11976.0   
2         25.0           11920.0   
3         25.0            9224.0   
4         25.0           11936.0   

   otp_read_line0_stdf_:ULT_Chksum@ULT_Chksum[1]  Part_Id  \
0                                            NaN      1.0   
1                                            NaN      2.0   
2                                            NaN      3.0   
3                                            NaN      4.0   
4                                            NaN      5.0   

   otp_read_blank:Functional[1]  g2bist_efuse_blank_read_f_:Functional[1]  \
0                           NaN                                       NaN   
1                           NaN                                       NaN   
2                           NaN                                       NaN   
3                           NaN                                       NaN   
4                

## Identifying the columns with less than 50,000 data points
For these columns, there are 2 ways of dealing with them - (i) to drop these columns completely and (ii) to replace the missing values

In [6]:
def identify_col(df):
    null_perCol = df.isnull().sum()
    insuff_col = {}
    for index in range(len(null_perCol)):
        if(df.shape[0]-null_perCol[index]<50000):
            insuff_col[null_perCol.index[index]] = null_perCol[index]
    return list(insuff_col.keys())

In [7]:
#Applying the above function to set of data
YK_less50000 = identify_col(YK_dropped)
#Printing to see the tests (columns) that has <50,000 data points 
print("For YK data: ",YK_less50000)

For YK data:  ['otp_read_line0_stdf_:ULT_Chksum@ULT_Chksum[1]', 'otp_read_blank:Functional[1]', 'g2bist_efuse_blank_read_f_:Functional[1]', 'tdf_50k_1_f_:Functional[1]', 'tdf_50k_2_f_:Functional[1]', 'stf_50k_1_f_:Functional[1]', 'tdf_50k_6_f_:Functional[1]', 'tdf_50k_3_f_:Functional[1]', 'g2bist_mac_256mhz_max_:Functional[1]', 'g2bist_mac_256mhz_min_:Functional[1]', 'sbist_soc_256mhz_max_:Functional[1]', 'sbist_soc_256mhz_min_:Functional[1]', 'stf_50k_2_f_:Functional[1]', 'stf_50k_3_f_:Functional[1]', 'stf_50k_4_f_:Functional[1]', 'tdf_50k_4_f_:Functional[1]', 'tdf_50k_5_f_:Functional[1]', 'tdf_50k_9_f_:Functional[1]', 'tdf_50k_8_f_:Functional[1]', 'tdf_90k_1_f_:Functional[1]', 'otp_read_line0_sva_f_:Functional[1]', 'tdf_50k_7_f_:Functional[1]', 'tdf_50k_10_f_:Functional[1]', 'tdf_90k_2_f_:Functional[1]', 'tdf_90k_3_f_:Functional[1]', 'tdf_90k_4_f_:Functional[1]', 'tdf_90k_5_f_:Functional[1]', 'tdf_90k_6_f_:Functional[1]', 'tdf_90k_7_f_:Functional[1]', 'tdf_90k_8_f_:Functional[1]', 't

The columns printed are those with < 50,000 data points. Since we fixed our sampling size to 50000 data points per test, we have to deal with these columns. However, note that some of these columns may not be applicable (can be removed completely) because it does not have a distribution (i.e the test will not appear in the label file). Hence, we need to identify the columns which are both applicable and have <50000 data points.

## Identify the applicable columns (i.e those with a label)
Those without a label will be dropped, since there will not be any response variable (distribution of categories) to predict the predictor variable (data points)

In [8]:
#Function to find out non applicable rows (no distribution, not part of training data)
def non_applicable (df_label):
    null_rowsLabel = df_label[df_label['Distribution Type'].isnull()]
    null_rowsLabel = list(null_rowsLabel['Name'])
    return null_rowsLabel

In [9]:
#Applying the above function to set of data
YK_non_applicable = non_applicable(YK_label)
#Printing the tests (columns) to see which tests do not have a distribution (i.e not applicable)
print(YK_non_applicable)
print(len(YK_non_applicable))

['otp_read_line0_stdf_:ULT_Chksum@ULT_Chksum[1]', 'otp_read_blank:Functional[1]', 'tdf_50k_1_f_:Functional[1]', 'tdf_50k_4_f_:Functional[1]', 'tdf_50k_9_f_:Functional[1]', 'tdf_50k_8_f_:Functional[1]', 'otp_read_line0_sva_f_:Functional[1]', 'tdf_50k_10_f_:Functional[1]', 'tdf_90k_2_f_:Functional[1]', 'tdf_90k_7_f_:Functional[1]', 'tdf_90k_8_f_:Functional[1]', 'tdf_90k_10_f_:Functional[1]', 'tdf_90k_11_f_:Functional[1]', 'tdf_90k_15_f_:Functional[1]']
14


## Identifying the applicable columns with <50,000 data points
These columns could either be dropped, or replace with data, depending on the nature of the columns/test itself.

In [10]:
#Function to find out the specific columns which we will need to inspect
def col_inspect(insuff_col_keys,null_rowsLabel):
    col_to_inspect = [i for i in insuff_col_keys if i not in null_rowsLabel]
    return col_to_inspect

The code above is to find the columns with <50,000 points, yet are applicable (i.e there is a label). This means that these are the columns we would need to inspect and do some pre processing to them so that there is sufficient data points (50,000) to train these test.

In [11]:
#Applying the above function to set of data
YK_inspect = col_inspect(YK_less50000,YK_non_applicable)
#Printing to see the cols to inspect
print(YK_inspect)

['g2bist_efuse_blank_read_f_:Functional[1]', 'tdf_50k_2_f_:Functional[1]', 'stf_50k_1_f_:Functional[1]', 'tdf_50k_6_f_:Functional[1]', 'tdf_50k_3_f_:Functional[1]', 'g2bist_mac_256mhz_max_:Functional[1]', 'g2bist_mac_256mhz_min_:Functional[1]', 'sbist_soc_256mhz_max_:Functional[1]', 'sbist_soc_256mhz_min_:Functional[1]', 'stf_50k_2_f_:Functional[1]', 'stf_50k_3_f_:Functional[1]', 'stf_50k_4_f_:Functional[1]', 'tdf_50k_5_f_:Functional[1]', 'tdf_90k_1_f_:Functional[1]', 'tdf_50k_7_f_:Functional[1]', 'tdf_90k_3_f_:Functional[1]', 'tdf_90k_4_f_:Functional[1]', 'tdf_90k_5_f_:Functional[1]', 'tdf_90k_6_f_:Functional[1]', 'tdf_90k_9_f_:Functional[1]', 'tdf_90k_12_f_:Functional[1]', 'tdf_90k_13_f_:Functional[1]', 'tdf_90k_14_f_:Functional[1]', 'tdf_90k_16_f_:Functional[1]', 'tdf_90k_17_f_:Functional[1]', 'tdf_90k_18_f_:Functional[1]']


## Replacing the missing values in these columns
Note that as shown by the printed output above, for YK data, all the columns to inspect (those applicable and with <50,000 data points), they are all functional tests. we would need to do another round of checks to find out the distributions (labels) of the tests above to ensure that the replacement of missing values does not affect the distribution. 

In [12]:
#Checking the distribution of the col shown above to inspect if there are really functional!
check_labels = YK_label[YK_label['Name'].isin(YK_inspect)]
check_labels['Distribution Type'].value_counts()

functional    26
Name: Distribution Type, dtype: int64

Since the above columns are all functional, we replace the missing values with 0 or 1 (since it will not affect the distribution)

In [13]:
#Function to replace the missing values with 0 and 1
def replace_func(df_dropped,df_col_inspect,non_applicable):
    pd.options.mode.chained_assignment = None 
    #Replacing missing 0 and 1 values
    for cols in df_col_inspect:
        null_count = df_dropped[cols].isnull().sum()
        df_dropped[cols] = df_dropped[cols].fillna(pd.Series(np.random.choice([0,-1],size=null_count)))
    #Dropping the tests with no distributions (not applicable)
    df = df_dropped.drop(columns=non_applicable)
    return df

In [14]:
#Applying it on data sets
YK_replace = replace_func(YK_dropped,YK_inspect,YK_non_applicable)

In [15]:
#Check if everything is replaced correctly (no more null values for the columns with <50000) data points
def check_nonull(df):
    null_perCol = df.isnull().sum()
    insuff_col = {}
    for index in range(len(null_perCol)):
        if(df.shape[0]-null_perCol[index]<50000):
            insuff_col[null_perCol.index[index]] = null_perCol[index]
    print(insuff_col)
    print(len(insuff_col))

In [16]:
check_nonull(YK_replace)

{}
0


## Sampling data
For columns with >50000 data points, systematic sampling + randomised sampling will be done. In intervals of 2, data points will be selected and the leftover points will be selected by random

In [17]:
#Converting systematic sampling (FOR FLOATS) to a function
def float_sample_2(population,sample_size):
    base_interval = len(population) // sample_size
    remainder = len(population)%sample_size
    #indices = []
    indices = {}
    #Add in min and max points in array
    min_index = population.argmin()
    max_index = population.argmax()
    indices[0] = min_index
    indices[1] = max_index
    start_index = np.random.randint(0,base_interval)
    for i in range(2,sample_size):
        interval = base_interval + 1 if i < remainder else base_interval
        #print("Interval is: ", interval)
        index = (start_index + i * interval) % len(population)
        #print("Index is: ",index)
        #indices.append(index)
        indices[i] = index
    #print("The dict is: ",indices)
    unique_indices = list(indices.values())
    unique_indices = [*set(unique_indices)]
    systematic_sample = population[unique_indices]
    print("Before delete")
    #print("systematic_sample: ",systematic_sample)
    population = np.delete(population,unique_indices)
    print("After delete")
    if len(systematic_sample) < sample_size:
        remaining_samples = sample_size - len(systematic_sample)
        remaining_indices = np.random.choice(len(population),remaining_samples,replace=False)
        systematic_sample = np.concatenate((systematic_sample,population[remaining_indices]))
    return systematic_sample

In [18]:
#Function to apply the systematic sampling for the function 
def apply_sampling(df):
    new_df = pd.DataFrame()
    for colName,colData in df.items():
        data = df.dropna(subset=colName)
        newCol = list(data[colName])
        newCol = np.array(newCol)
        newCol = pd.Series(float_sample_2(newCol,50000))
        #Can only concat after sampling!!
        new_df = pd.concat([new_df,newCol.rename(colName)],axis=1)
    return new_df

In [19]:
#Apply sampling functions to datasets
YK_data_final = apply_sampling(YK_replace)

Before delete
After delete
Before delete
After delete
Before delete
After delete
Before delete
After delete
Before delete
After delete
Before delete
After delete
Before delete
After delete
Before delete
After delete
Before delete
After delete
Before delete
After delete
Before delete
After delete
Before delete
After delete
Before delete
After delete
Before delete
After delete
Before delete
After delete
Before delete
After delete
Before delete
After delete
Before delete
After delete
Before delete
After delete
Before delete
After delete
Before delete
After delete
Before delete
After delete
Before delete
After delete
Before delete
After delete
Before delete
After delete
Before delete
After delete
Before delete
After delete
Before delete
After delete
Before delete
After delete
Before delete
After delete
Before delete
After delete
Before delete
After delete
Before delete
After delete
Before delete
After delete
Before delete
After delete
Before delete
After delete
Before delete
After delete
B

Before delete
After delete
Before delete
After delete
Before delete
After delete
Before delete
After delete
Before delete
After delete
Before delete
After delete
Before delete
After delete
Before delete
After delete
Before delete
After delete
Before delete
After delete
Before delete
After delete
Before delete
After delete
Before delete
After delete
Before delete
After delete
Before delete
After delete
Before delete
After delete
Before delete
After delete
Before delete
After delete
Before delete
After delete
Before delete
After delete
Before delete
After delete
Before delete
After delete
Before delete
After delete
Before delete
After delete
Before delete
After delete
Before delete
After delete
Before delete
After delete
Before delete
After delete
Before delete
After delete
Before delete
After delete
Before delete
After delete
Before delete
After delete
Before delete
After delete
Before delete
After delete
Before delete
After delete
Before delete
After delete
Before delete
After delete
B

Before delete
After delete
Before delete
After delete
Before delete
After delete
Before delete
After delete
Before delete
After delete
Before delete
After delete
Before delete
After delete
Before delete
After delete
Before delete
After delete
Before delete
After delete
Before delete
After delete
Before delete
After delete
Before delete
After delete
Before delete
After delete
Before delete
After delete
Before delete
After delete
Before delete
After delete
Before delete
After delete
Before delete
After delete
Before delete
After delete
Before delete
After delete
Before delete
After delete
Before delete
After delete
Before delete
After delete
Before delete
After delete
Before delete
After delete
Before delete
After delete
Before delete
After delete
Before delete
After delete
Before delete
After delete
Before delete
After delete
Before delete
After delete
Before delete
After delete
Before delete
After delete
Before delete
After delete
Before delete
After delete
Before delete
After delete
B

Before delete
After delete
Before delete
After delete
Before delete
After delete
Before delete
After delete
Before delete
After delete
Before delete
After delete
Before delete
After delete
Before delete
After delete
Before delete
After delete
Before delete
After delete
Before delete
After delete
Before delete
After delete
Before delete
After delete
Before delete
After delete
Before delete
After delete
Before delete
After delete
Before delete
After delete
Before delete
After delete
Before delete
After delete
Before delete
After delete
Before delete
After delete
Before delete
After delete
Before delete
After delete
Before delete
After delete
Before delete
After delete
Before delete
After delete
Before delete
After delete
Before delete
After delete
Before delete
After delete
Before delete
After delete
Before delete
After delete
Before delete
After delete
Before delete
After delete
Before delete
After delete
Before delete
After delete
Before delete
After delete
Before delete
After delete
B

## Normalisation of the data 
Data are normalised using the min max module from scikit learn. This will compress the data points from 0 to 1 but will not change its distribution.

In [20]:
#Import relevant packages 
from sklearn import preprocessing
min_max = preprocessing.MinMaxScaler()

In [21]:
#Function to normalise all the columns in a dataframe
def normalise(df):
    final_df = pd.DataFrame()
    for colName,colData in df.items():
        c_scaled = min_max.fit_transform(df[[colName]])
        c_scaled = pd.Series(c_scaled.ravel())
        final_df = pd.concat([final_df,c_scaled.rename(colName)],axis=1)
    return final_df

In [22]:
#Applying it to the datasets
YK_data_final = normalise(YK_data_final)
#Printing to see the shape of the data
print(YK_data_final.shape)
print(YK_data_final.head())

(50000, 1025)
   temperature  Device_Test_Time   Part_Id  \
0          0.0          0.399193  0.000000   
1          0.0          0.397441  0.000206   
2          0.0          0.396992  0.000308   
3          0.0          0.586461  0.000411   
4          0.0          0.586461  0.000514   

   g2bist_efuse_blank_read_f_:Functional[1]  tdf_50k_2_f_:Functional[1]  \
0                                       0.0                         1.0   
1                                       1.0                         0.0   
2                                       1.0                         0.0   
3                                       0.0                         0.0   
4                                       1.0                         0.0   

   stf_50k_1_f_:Functional[1]  write_dro_sidd:Functional[1]  \
0                         1.0                           0.0   
1                         0.0                           0.0   
2                         0.0                           0.0   
3     

## Pre-processing the label file
The codes above are for the raw data (The data points for each test).The labels under the label file (YK_label) are the categorical variables denoting the distribution of each test (each column in the raw data file). There are 6 labels/distribution categories - outlier, normal, longtail, functional, discrete and bimodal. Now we will process the label files and identify those rows where there are no distribution (not applicable) and proceed to drop them. 

In [23]:
#Function to faciliate the dropping of the null rows in labels file
def drop_null_labels(label_df,not_applicable):
    df = label_df.drop(label_df[label_df.Name.isin(not_applicable)].index)
    return df

In [24]:
#Applying it to set of data
YK_label_final = drop_null_labels(YK_label,YK_non_applicable)

Now for visualisation, we will concatenate the dataframes together, such that we will have 2 columns, 'Name' and 'Distribution Type' for the tests and distribution respectively. We will use the below 2 functions to facilitate the concatenation.

In [25]:
#Function to concat the dataframes (raw data and the name of the tests) -> row format. i.e 2 columns: data, name
def concat_xy(df):
    cols = ['Data','Name']
    concat_df = pd.DataFrame(columns=cols,index=range(df.shape[1]))
    index = 0
    for colName, colData in df.items():
        #variable index is to retrieve the index of the columns in YK raw (final_df)
        vals = np.array(colData.values)
        concat_df.loc[index].Data = vals
        concat_df.loc[index].Name = colName
        index+=1
    return concat_df

In [26]:
#Applying it to our dataset
YK_testname_data = concat_xy(YK_data_final)
#See how it looks like
print(YK_testname_data.head())

                                                Data  \
0  [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...   
1  [0.399193055385744, 0.39744059991033953, 0.396...   
2  [0.0, 0.0002055076037813399, 0.000308261405672...   
3  [0.0, 1.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0, ...   
4  [1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0, ...   

                                       Name  
0                               temperature  
1                          Device_Test_Time  
2                                   Part_Id  
3  g2bist_efuse_blank_read_f_:Functional[1]  
4                tdf_50k_2_f_:Functional[1]  


In [27]:
#Function concatenating the Y (distribution) with the raw data (X) -> dataframe passed in is the label final! concat_df is testname_data
def model_xy(label_df,concat_df):
    dist_labels = label_df[['Distribution Type','Name']]
    model_df = pd.merge(concat_df,dist_labels,on='Name')
    model_df_xy = model_df[['Data','Distribution Type']]
    return model_df_xy

In [28]:
#Applying it to the data sets
YK_model = model_xy(YK_label_final,YK_testname_data)
print(YK_model.shape)
#Visualising how it looks like
print(YK_model.head())

(1025, 2)
                                                Data Distribution Type
0  [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...        functional
1  [0.399193055385744, 0.39744059991033953, 0.396...          longtail
2  [0.0, 0.0002055076037813399, 0.000308261405672...          longtail
3  [0.0, 1.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0, ...        functional
4  [1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0, ...        functional


## Exporting our data 
We have to export our data in 2 separate files (raw data) and the labels before using the above functions to concatenate them together for modelling. This is because excel does not support arrays within each cell. This is exacerbated by the fact that our numbers are floating point numbers, and hence conversion of these numbers to strings may cause it to lose precision as well.

In [29]:
#Final check before exporting to csv
print(YK_data_final.shape)
print(check_nonull(YK_data_final))
print(YK_label_final.shape)
print(YK_label_final['Distribution Type'].isnull().sum())

(50000, 1025)
{}
0
None
(1025, 4)
0


In [30]:
#Export the raw data file (consisting of 50,000 data points for each test)
YK_data_final.to_csv(r"C:\Users\nxg00371\Desktop\Coding Projs\YK_cleaned_rawdata.csv",index=False)
#Export the labels file (consisting of the distribution of each test)
YK_label_final.to_csv(r"C:\Users\nxg00371\Desktop\Coding Projs\YK_cleaned_labels.csv",index=False)